In [2]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [3]:
!python -V

Python 3.11.9


In [19]:
import pickle
import pandas as pd
import os

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [13]:
df = read_data('/home/deepak/Documents/mlops_zoomcamp/mlops-zoomcamp/homework-3/mage_data/yellow_tripdata_2023-03.parquet')

In [14]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [15]:
y_pred.std()

6.247488852238703

In [16]:
# Create an artificial ride_id column
year = 2023  # Replace with the actual year if needed
month = 3    # Replace with the actual month if needed
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [17]:
# Create a DataFrame with the results
df_result = df[['ride_id']].copy()
df_result['predicted_duration'] = y_pred

# Save the results as a parquet file
output_file = 'predictions.parquet'  # Replace with your desired output file path
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [20]:
# Get the size of the output file
file_size = os.path.getsize(output_file)
print(f"The size of the output file is {file_size} bytes.")

The size of the output file is 68641880 bytes.
